# Financial Econometrics I: Homework 2
Jan Kubal & Matyáš Mattanelli

### Problem 1

In [2]:
#Loading necessary packages
suppressPackageStartupMessages({
library(quantmod)
})

In [3]:
#Loading the file containing the symbols
symbols2 <- read.csv("symbols2.csv")

#Generating 100 random numbers
set.seed(48983854) #random seed using the SIS number
indices <- sort(sample(1:377, 100, replace = F))

#Extracting the tickers
tickers <- symbols2$Symbol[indices]

#Downloading the data for each ticker
tickers_data <- vector("list", length(tickers)) #Empty list for the data
options("getSymbols.warning4.0" = FALSE) #To suppress an annoying message
for (i in 1:length(tickers)) {
    tickers_data[[i]] <- getSymbols(tickers[i], src = "yahoo", auto.assign = F, from = "2019-01-01", to = "2022-01-01")
}

Warning message:
"BMS contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them."


In [29]:
#check that closing price is in the fourth column
fourth_cols <- vector("character", length(tickers_data))
is_close <- vector("character", length(tickers_data))

for(i in 1:length(tickers_data)) {
 fourth_cols[i] <- names(tickers_data[[i]])[4]
}

for(i in 1:length(tickers_data)) {
 is_close[i] <- substr(fourth_cols[i],  nchar(fourth_cols[i])-4, nchar(fourth_cols[i]))
}

sum(is_close == "Close") #If we get here sum of 100, we got all the closing prices in the fourth column and can continue to subsetting

[1] 100

In [23]:
fourth_cols[1]

[1] "ABT.Close"

In [16]:
names(tickers_data[[2]])[4]

[1] "ACN.Close"